In [ ]:
!pip install azure-storage-blob
!pip install boto3
!pip install google_cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.2 MB/s eta 0:00:00


In [7]:
# import librairies
import pandas as pd
import numpy as np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, ContainerClient
from google.cloud import storage
from io import StringIO

In [ ]:
#import data
URL = "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/4fc8e993-c3b9-4eb2-b9bb-dfbde9b1fb6f/download/public_up_to_150k_8_230930.csv"
df_raw=pd.read_parquet(URL, engine='pyarrow')
df_raw.head()

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
print(df_raw.shape)# to know how many row and col we have

In [ ]:
print(df_raw.columns) # know col name

In [ ]:
#drop some na value if we have
df_raw.dropna()
df_raw.shape

In [ ]:
#checking if we successfull drop na value
df_raw.info()

In [11]:
# Function

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

# Google Cloud Functions
def google_upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

def google_download_blob(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

# AWS Functions
def aws_upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(file_name, bucket, object_name)
    print(f"Uploaded {file_name} to S3 bucket {bucket}.")

def aws_download_file(bucket, object_name, file_name):
    s3_client = boto3.client('s3')
    s3_client.download_file(bucket, object_name, file_name)
    print(f"Downloaded {object_name} from S3 bucket {bucket}.")

In [13]:
import pandas as pd
import json
from io import StringIO
from azure.storage.blob import BlobServiceClient


# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Azure Blob Storage configuration
CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'projectdata'

# List of data source URLs
data_source_urls = [
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/a7fa66f4-fd2e-433c-8ef9-59780ef60ae5/download/public_up_to_150k_1_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/7d2308a8-0ac1-48a8-b21b-f9eb373ac417/download/public_up_to_150k_2_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/5158aae1-066d-4d01-a226-e44ecc9bdda7/download/public_up_to_150k_3_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/d888bab1-da5b-46f2-bed2-a052d48af246/download/public_up_to_150k_4_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/ee12d751-2bb4-4343-8330-32311ae4e7c7/download/public_up_to_150k_5_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/27b874d9-a059-4296-bb74-374294c48616/download/public_up_to_150k_6_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/434efae0-016a-48da-92dc-c6f113d827c1/download/public_up_to_150k_7_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/4fc8e993-c3b9-4eb2-b9bb-dfbde9b1fb6f/download/public_up_to_150k_8_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/7f9c6867-2b55-472e-a4f3-fd0f5f27f790/download/public_up_to_150k_9_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/a8f2c8b2-facb-4e97-ad5f-7c8736c8b4b6/download/public_up_to_150k_10_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/6f9787a3-afd6-45b2-b78e-ad0dc097c1c3/download/public_up_to_150k_11_230930.csv",
    "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/b6528428-fbd9-4ca6-ae08-9e3416f8ee7f/download/public_up_to_150k_12_230930.csv"
]

for idx, data_source_url in enumerate(data_source_urls, start=1):
    # Download data from the URL and load it into a DataFrame
    df_raw = pd.read_csv(data_source_url)

    # Convert DataFrame to CSV
    output = StringIO()
    df_raw.to_csv(output, index=False)
    data = output.getvalue()
    output.close()

    # Blob name based on index
    blob_name = f"public_up_to_150k_{idx}_230930.csv"

    # Create the BlobServiceClient object
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

    # Get a blob client using the container name and blob name
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

    # Upload the CSV data
    blob_client.upload_blob(data, overwrite=True)

    print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")


Uploaded public_up_to_150k_1_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_2_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_3_230930.csv to Azure Blob Storage in container projectdata.


<ipython-input-13-9055b598ce06>:35: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(data_source_url)


Uploaded public_up_to_150k_4_230930.csv to Azure Blob Storage in container projectdata.


<ipython-input-13-9055b598ce06>:35: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(data_source_url)


Uploaded public_up_to_150k_5_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_6_230930.csv to Azure Blob Storage in container projectdata.


<ipython-input-13-9055b598ce06>:35: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(data_source_url)


Uploaded public_up_to_150k_7_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_8_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_9_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_10_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_11_230930.csv to Azure Blob Storage in container projectdata.
Uploaded public_up_to_150k_12_230930.csv to Azure Blob Storage in container projectdata.


In [ ]:
import zipfile

# Download the ZIP file
url = "https://apps.bea.gov/regional/zip/SQGDP.zip"
response = requests.get(url)

# Save the ZIP file
zip_file_path = "SQGDP.zip"
with open(zip_file_path, "wb") as f:
    f.write(response.content)

# Extract the CSV file from the ZIP archive
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    csv_file_name = "SQGDP2__ALL_AREAS_2018_2023.csv"
    zip_ref.extract(csv_file_name)

# Parse the CSV file using pandas
df = pd.read_csv(csv_file_name)

config_file_path = 'config.json'

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Upload the CSV file to Azure Blob Storage
connection_string = config['connectionString']
container_name = "projectdata"
blob_name = "SQGDP2__ALL_AREAS_2018_2023.csv"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

with open(csv_file_name, "rb") as data:
    container_client.upload_blob(name=blob_name, data=data)

print("CSV file uploaded to Azure Blob Storage.")

In [ ]:
url = "https://www.census.gov/naics/2022NAICS/2022_NAICS_Descriptions.xlsx"
response = requests.get(url)

# Save the Excel file
excel_file_path = "2022_NAICS_Descriptions.xlsx"
with open(excel_file_path, "wb") as f:
    f.write(response.content)

# Parse the Excel file using pandas
df = pd.read_excel(excel_file_path)

config_file_path = 'config.json'

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Upload the CSV file to Azure Blob Storage
connection_string = config['connectionString']
container_name = "projectdata"
blob_name = "2022_NAICS_Descriptions.xlsx"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

with open(excel_file_path, "rb") as data:
    container_client.upload_blob(name=blob_name, data=data)

print("Excel file uploaded to Azure Blob Storage.")

#Downloading data from Azure Container

In [ ]:
config_file_path = 'config.json'

#load the JSON configuration file
with open(config_file_path, 'r') as config_file:
  config = json.load(config_file)

#print the configuration
#Connection_STRING = config['CONNECTION_STRING_AZURE_STORAGE']

CONNECTION_STRING_AZURE_STORAGE = config['connectionString']
CONTAINER_AZURE = 'projectdata'
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)
ppp_data = []
gdp_data = []
naics = []
blob_list = container_client.list_blobs()
for blob in blob_list:
  if blob.name == "2022_NAICS_Descriptions.xlsx" or blob.name == "SQGDP2__ALL_AREAS_2018_2023.csv":
    if blob.name == "2022_NAICS_Descriptions.xlsx":
      blob_client = container_client.get_blob_client(blob = blob.name)
      blob_data = blob_client.download_blob()
      blob_content = blob_data.readall()
      df = pd.read_excel(blob_content)
      naics.append(df)
    elif blob.name == "SQGDP2__ALL_AREAS_2018_2023.csv":
      blob_client = container_client.get_blob_client(blob = blob.name)
      blob_data = blob_client.download_blob()
      blob_content = blob_data.readall().decode('utf-8')
      df1 = pd.read_csv(StringIO(blob_content))
      gdp_data.append(df1)
  else:
    blob_client = container_client.get_blob_client(blob = blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df2 = pd.read_csv(StringIO(blob_content))
    ppp_data.append(df2)

In [ ]:
all_naics = pd.concat(naics, ignore_index=True)

In [ ]:
all_ppp = pd.concat(ppp_data, ignore_index=True)

In [ ]:
all_gdp = pd.concat(gdp_data, ignore_index=True)